In [1]:
!pip install -q easyocr
!pip install deepface
!pip install groq
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 11.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=b7e8c71d3cf2c2d457681090935f0ee1f44f9721bbf414f274fd20357545fbe1
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
     ━━━━━━━━━━━━━━━━━━━━━━━

Các bạn lấy API key của Groq tại link này: [https://console.groq.com/keys](https://console.groq.com/keys)

In [2]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.049s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [3]:
%%writefile app.py
import streamlit as st
import easyocr
import numpy as np
from PIL import Image
import requests
import json
from transformers import pipeline
from groq import Groq
from deepface import DeepFace
import cv2
import os

# Initialize the OCR reader
ocr_reader = easyocr.Reader(["en"])

def get_image_caption(image):
    # Use a pre-trained image captioning model from Salesforce
    caption_pipeline = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")
    return caption_pipeline(image)[0]['generated_text']

def perform_ocr(image):
    result = ocr_reader.readtext(np.array(image))
    ocr_texts = [line[1] for line in result]
    return ocr_texts

def people_analysis(image):
    # Use DeepFace to perform face analysis
    faces = DeepFace.analyze(
        img_path = image,
        detector_backend = "retinaface",
        enforce_detection = False,
        actions = ['age', 'gender', 'emotion']
    )
    return faces

def analyze_image_information(image_description, ocr_results, pp_anal):
    prompt = f"""
    Analyze the following image information and provide insights based on the criteria given below:

    Image Description:
    {image_description}

    OCR Results:
    {ocr_results}

    Face analysis results:
    {pp_anal}

    Criteria:
    1. Brand Logos: Identify any brand logos mentioned in the description or OCR results.
    2. Products: Mention any products such as beer kegs and bottles.
    3. Customers: Describe the number of customers, their activities, and emotions. You don't need to texplain about those number appear in the content.
    4. Promotional Materials: Identify any posters, banners, and billboards.
    5. Setup Context: Determine the scene context (e.g., bar, restaurant, grocery store, or supermarket).

    Insights:
    """

    # Replace with your Groq API key

    client = Groq(
        # This is the default and can be omitted
        api_key="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",
    )


    data = {
        "model": "llama3-70b-8192",
        "messages": [{"role": "user", "content": prompt}]
    }

    chat_completion = client.chat.completions.create(**data)
    return chat_completion.choices[0].message.content


# Streamlit app
st.set_page_config(layout="wide")
st.title("Image Analysis App")

# Create three columns with custom widths
col1, col2, col3 = st.columns([1, 2, 2])

with col1:
    st.header("Upload Image")
    uploaded_file = st.file_uploader("Choose an image file", type=["jpg", "jpeg", "png"])

with col2:
    st.header("OCR and Description")

    if uploaded_file is not None:
        # Load the image
        image = Image.open(uploaded_file).convert("RGB")
        st.image(image, caption='Uploaded Image', use_column_width=True)

        # Get image caption
        st.subheader("Image Description")
        image_description = get_image_caption(image)
        st.write(image_description)

        # # Perform OCR
        # st.subheader("OCR Texts")
        ocr_texts = perform_ocr(image)
        # for text in ocr_texts:
        #     st.write(text)

        # Perform face analysis
        st.subheader("Face Analysis")
        image = np.array(image)
        faces = people_analysis(image)

        # Duyệt qua từng kết quả inference và vẽ thông tin lên ảnh
        for result in faces:
            region = result['region']
            if(result['emotion']['happy'] > result['emotion']['neutral']):
                emotion = 'happy'
            else:
                emotion = 'neurral'
            dominant_gender = result['dominant_gender']
            age = result['age']

            # Tọa độ bounding box được phóng to lên 10 lần
            top_left = (region['x'], region['y'])
            bottom_right = (region['x']+region['w'], region['y']+region['h'])

            # Vẽ khung hình chữ nhật quanh khuôn mặt
            cv2.rectangle(image, top_left, bottom_right, (0, 0, 255), 10)

            # Ghi thông tin lên ảnh
            text = f"Age: {age}\nGender: {dominant_gender}\nEmotion: {emotion}"
            y0, dy = region['y'] - 100, 50
            for i, line in enumerate(text.split('\n')):
                y = y0 + i*dy
                cv2.putText(image, line, (region['x'], y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 8, cv2.LINE_AA)

        st.image(image, caption='Facial Emotion Image', use_column_width=True)


with col3:
    st.header("Analysis")

    if uploaded_file is not None:
        # Analyze image information
        ocr_results = ' '.join(ocr_texts)
        pp_anal = ''.join([f"Age: {face['age']}, Gender: {face['dominant_gender']}, Emotion: {face['dominant_emotion']}\n" for face in faces])
        analysis = analyze_image_information(image_description, ocr_results, pp_anal)
        st.write(analysis)

Writing app.py


In [4]:
!streamlit run app.py &>/content/logs.txt &

In [5]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.19.75.22


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 1.78s
your url is: https://short-pears-heal.loca.lt
^C
